In [2]:
# Hugging Face Transformers and PyTorch
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, RobertaForSequenceClassification, RobertaTokenizer
from torch.utils.data import TensorDataset, DataLoader


# Sklearn for Model Evaluation
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import numpy as np


In [3]:
# Load the pre-trained model and tokenizer from Hugging Face
model_name = "AleOfDurin/Sentiment_fine_tuned"
# Load the model and tokenizer from the Hugging Face Hub
model = RobertaForSequenceClassification.from_pretrained(model_name) # Ensure it's SequenceClassification model  --- ,num_labels=2) deleted as it was added to the json file
tokenizer = RobertaTokenizer.from_pretrained(model_name)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at AleOfDurin/Sentiment_fine_tuned and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Load the tokenized validation dataset (save the file to the path desired and link it here)
val_encodings, val_labels = torch.load(r'C:\Users\aless\Desktop\final project 2.1\val_encodings.pt')


C:\Users\aless\AppData\Local\Temp\ipykernel_282536\861193737.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  val_encodings, val_labels = torch.load(r'C:\Users\aless\Desk

In [5]:
# Define the device: Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the GPU
model.to(device)

# Set batch size and number of workers
batch_size = 512
num_workers = 8

# Create the validation DataLoader with batch size 512 and workers set to 8
val_dataset = TensorDataset(
    val_encodings['input_ids'].to(device),  # Move input IDs to GPU
    val_encodings['attention_mask'].to(device),  # Move attention masks to GPU
    val_labels.to(device)  # Move labels to GPU
)
val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=num_workers)

# Set the model to evaluation mode
model.eval()

print("Model and data moved to GPU, DataLoader set.")


Model and data moved to GPU, DataLoader set.


In [6]:
model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(0.3),  # Dropout as per the trained model
    torch.nn.Linear(model.config.hidden_size, 2)  # Linear layer for binary classification
).to(device) 

In [7]:
# Example: Re-tokenizing the validation dataset (assuming you have the raw text data)
texts = ["Your sample text here", "Another sample text"]  # Replace with your dataset's text

# Tokenize the texts
tokenized_inputs = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")

# Move tokenized inputs to the device
input_ids = tokenized_inputs['input_ids'].to(device)
attention_mask = tokenized_inputs['attention_mask'].to(device)

# Debug: Print tokenized inputs
print(f"Tokenized Input IDs: {input_ids}")
print(f"Tokenized Attention Mask: {attention_mask}")


Tokenized Input IDs: tensor([[    0, 12861,  7728,  2788,   259,     2],
        [    0, 21518,  7728,  2788,     2,     1]], device='cuda:0')
Tokenized Attention Mask: tensor([[1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 0]], device='cuda:0')


In [8]:
import time

# Start timer for inference
start_time = time.time()

# Initialize lists to store predictions and true labels
all_preds = []
all_labels = []
y_proba = []  # Store probabilities for ROC-AUC

# Set model to no_grad mode for inference
with torch.no_grad():
    # Loop through validation DataLoader
    for step, batch in enumerate(val_loader):
        # Move batch to the device (GPU or CPU)
        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        # Check for NaN values in the input data
        if torch.isnan(input_ids).any() or torch.isnan(attention_mask).any():
            print("Input contains NaN values.")
            break  # Stop if NaNs are found
        
        # Remove mixed precision and run in full precision
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        
        # Use only the logits for the [CLS] token (first token) for classification
        cls_logits = logits[:, 0, :]  # Extract the [CLS] token logits for each sequence
        
        # Predicted classes from [CLS] token
        predictions = torch.argmax(cls_logits, dim=-1)
        
        # Probabilities for ROC-AUC from [CLS] token
        probabilities = torch.softmax(cls_logits, dim=-1)[:, 1]  # Probabilities for class 1

        # Store predictions and true labels
        all_preds.extend(predictions.cpu().numpy())  # Store binary predictions for each sequence
        all_labels.extend(labels.cpu().numpy())  # Store actual labels
        y_proba.extend(probabilities.cpu().numpy())  # Store probabilities for ROC-AUC

# Total time taken for inference
total_time = time.time() - start_time
print(f"Total inference time: {total_time:.2f} seconds")


c:\Users\aless\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\roberta\modeling_roberta.py:370: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Total inference time: 568.10 seconds


In [9]:
# Check the shape and some example values
print(f"Predictions (all_preds): {all_preds[:5]}")  # Print first 5 predictions
print(f"Labels (all_labels): {all_labels[:5]}")  # Print first 5 labels


Predictions (all_preds): [0, 0, 0, 0, 0]
Labels (all_labels): [0, 0, 0, 0, 0]


In [10]:

# Check distribution of predictions (if the model is predicting only one class)
unique_preds, counts_preds = np.unique(all_preds, return_counts=True)
pred_class_distribution = dict(zip(unique_preds, counts_preds))
print(f"Predicted Class Distribution: {pred_class_distribution}")

Predicted Class Distribution: {0: 469723}


In [11]:
all_preds_flat = np.concatenate([pred.flatten() for pred in all_preds])
all_labels_flat = np.concatenate([label.flatten() for label in all_labels])
y_proba_flat = np.array(y_proba).flatten()


In [12]:
# Calculate Accuracy
accuracy = accuracy_score(all_labels_flat, all_preds_flat)
print(f"Accuracy: {accuracy:.4f}")

# Generate Classification Report (Precision, Recall, F1-Score)
report = classification_report(all_labels_flat, all_preds_flat)
print(report)

# Confusion Matrix
conf_matrix = confusion_matrix(all_labels_flat, all_preds_flat)
print("Confusion Matrix:")
print(conf_matrix)

# ROC-AUC Score
roc_auc = roc_auc_score(all_labels_flat, y_proba_flat)
print(f"ROC-AUC Score: {roc_auc:.4f}")

Accuracy: 1.0000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    469723

    accuracy                           1.00    469723
   macro avg       1.00      1.00      1.00    469723
weighted avg       1.00      1.00      1.00    469723

Confusion Matrix:
[[469723]]


c:\Users\aless\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


ValueError: Input contains NaN.